<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/modelB/cnnTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model (1).py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 256
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [8]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

In [9]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch, lr):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best_"+str(lr)+".pt")
        try:
          if gsync_save:
              gsync.update_file_to_folder("best_"+str(lr)+".pt")
        except:
          print('Failed to gsync_save.')

        try:
          model_save_name = 'best_'+str(lr)
          path = F"/content/drive/My Drive/dl-reproducibility-project/model/{model_save_name}" 
          torch.save(model.state_dict(), path+'.epoch-{}.pt'.format(epoch))
        except:
          print('Failed to save best model to personal google drive')

    return best_loss, best_epoch

In [12]:
try:
  os.makedirs('/content/drive/My Drive/dl-reproducibility-project/model/')
except:
  print('')

In [13]:
for lr in [0.25,0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model(basemodel=[False, True, False], modifiedModel=[True, False, False, False])
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch = test(epoch, best_loss, best_epoch, lr)

  # if in Google Colab, download your model with this
  #from google.colab import files
  #files.download("best_"+str(lr)+"pt")
  

Training with  0.25


Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=False)
    (5): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Dropout(p=0.5, inplace=False)
    (9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (12): ReLU()
    (13): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (14): ReLU()
    (15): AdaptiveAvgPool2d(output_size=1)
    (16): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
 

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.304143
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.210622

Test set: Average loss: 0.0172, Accuracy: 2254/10000 (23%)



Uploading file best_0.25.pt: 100%|██████████| 100/100 [00:01<00:00, 71.23it/s]

Failed to save best model to personal google drive


...

Failed to save best model to personal google drive


Train Epoch: 149 [0/50000 (0%)]	Loss: 0.505430
Train Epoch: 149 [25600/50000 (51%)]	Loss: 0.557475

Test set: Average loss: 0.0051, Accuracy: 7943/10000 (79%)

Train Epoch: 150 [0/50000 (0%)]	Loss: 0.564846
Train Epoch: 150 [25600/50000 (51%)]	Loss: 0.607666

Test set: Average loss: 0.0051, Accuracy: 7884/10000 (79%)

Train Epoch: 151 [0/50000 (0%)]	Loss: 0.463948
Train Epoch: 151 [25600/50000 (51%)]	Loss: 0.532321

Test set: Average loss: 0.0052, Accuracy: 7906/10000 (79%)

Train Epoch: 152 [0/50000 (0%)]	Loss: 0.518717
Train Epoch: 152 [25600/50000 (51%)]	Loss: 0.437082

Test set: Average loss: 0.0053, Accuracy: 7902/10000 (79%)

Train Epoch: 153 [0/50000 (0%)]	Loss: 0.606260
Train Epoch: 153 [25600/50000 (51%)]	Loss: 0.586536

Test set: Average loss: 0.0054, Accuracy: 7893/10000 (79%)

Train Epoch: 154 [0/50000 (0%)]	Loss: 0.521373
Train Epoch: 154 [25600/50000 (51%)]	Loss: 0.510795

Test set: Average loss: 0.0054, Accuracy: 7896/10000 (79%)

Train Epoch: 155 [0/50000 (0%)]	Loss: 0.

Uploading file best_0.01.pt: 100%|██████████| 100/100 [00:00<00:00, 113.85it/s]

Failed to save best model to personal google drive


Train Epoch: 173 [0/50000 (0%)]	Loss: 0.625483
Train Epoch: 173 [25600/50000 (51%)]	Loss: 0.605190

Test set: Average loss: 0.0054, Accuracy: 7838/10000 (78%)

Train Epoch: 174 [0/50000 (0%)]	Loss: 0.419896
Train Epoch: 174 [25600/50000 (51%)]	Loss: 0.500190

Test set: Average loss: 0.0050, Accuracy: 8022/10000 (80%)

Train Epoch: 175 [0/50000 (0%)]	Loss: 0.478249
Train Epoch: 175 [25600/50000 (51%)]	Loss: 0.405354

Test set: Average loss: 0.0051, Accuracy: 7954/10000 (80%)

Train Epoch: 176 [0/50000 (0%)]	Loss: 0.560818
Train Epoch: 176 [25600/50000 (51%)]	Loss: 0.577619

Test set: Average loss: 0.0051, Accuracy: 7946/10000 (79%)

Train Epoch: 177 [0/50000 (0%)]	Loss: 0.508859
Train Epoch: 177 [25600/50000 (51%)]	Loss: 0.489370

Test set: Average loss: 0.0051, Accuracy: 8001/10000 (80%)

Train Epoch: 178 [0/50000 (0%)]	Loss: 0.557265
Train Epoch: 178 [25600/50000 (51%)]	Loss: 0.500753

Test set: Average loss: 0.0050, Accuracy: 8031/10000 (80%)

Train Epoch: 179 [0/50000 (0%)]	Loss: 0.